In [2]:
import cv2
import face_recognition
import numpy as np
import os
import matplotlib.pyplot as plt

# **Landmark Detection**

In [5]:
sift = cv2.SIFT_create()

bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

In [6]:
landmark_dir = 'content/museum_images'

landmark_descriptors = []
landmark_keypoints = []
landmark_images = []

for img_name in os.listdir(landmark_dir):
    img_path = os.path.join(landmark_dir, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue
    img = cv2.resize(img, (600, 400))
    kp, des = sift.detectAndCompute(img, None)
    if des is not None:
        landmark_keypoints.append(kp)
        landmark_descriptors.append(des)
        landmark_images.append(img)

In [2]:
def detect_landmark(query_img, match_threshold=10):
    if query_img is None:
        return query_img
    query_img = cv2.resize(query_img, (600, 400))
    query_kp, query_des = sift.detectAndCompute(query_img, None)

    best_match_count = 0
    best_good_matches = []
    best_index = -1
    best_homography = None
    best_pts = None

    for i, des in enumerate(landmark_descriptors):
        matches = bf.match(des, query_des)
        matches = sorted(matches, key=lambda x: x.distance)
        good_matches = matches[:50]

        if len(good_matches) >= match_threshold:
            src_pts = np.float32([landmark_keypoints[i][m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([query_kp[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

            H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
            matches_mask = mask.ravel().tolist()
            inliers = sum(matches_mask)

            if H is not None and inliers > best_match_count:
                best_match_count = inliers
                best_good_matches = good_matches
                best_index = i
                best_homography = H

    if best_match_count >= match_threshold:
        print(f"✅ Landmark detected with {best_match_count} inlier matches.")

        matched_img = cv2.drawMatches(
            landmark_images[best_index], landmark_keypoints[best_index],
            query_img, query_kp,
            best_good_matches, None,
            matchColor=(0, 255, 0),
            singlePointColor=None,
            matchesMask=[1]*len(best_good_matches),
            flags=2
        )

        return matched_img
    else:
        return query_img


Landmark Detection on Still Image

In [7]:
query_img = cv2.imread('content/query.jpg', cv2.IMREAD_GRAYSCALE)
matched_img = detect_landmark(query_img)
cv2.imshow('Matched Image', matched_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

✅ Landmark detected with 32 inlier matches.


Real-Time Landmark Detection

In [12]:
cam = cv2.VideoCapture(0)

while True:
    ret, frame = cam.read()
    if not ret:
        break
    result = detect_landmark(frame, match_threshold=10)
    cv2.imshow("Landmark Detection", result)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cam.release()
cv2.destroyAllWindows()


✅ Landmark detected with 15 inlier matches.
✅ Landmark detected with 46 inlier matches.
✅ Landmark detected with 44 inlier matches.
✅ Landmark detected with 48 inlier matches.
✅ Landmark detected with 45 inlier matches.
✅ Landmark detected with 43 inlier matches.
✅ Landmark detected with 49 inlier matches.


# **Face Recognition**

In [8]:
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

Create Encoding of Known Faces

In [9]:
known_face_encodings = []
known_face_names = []

path = "known_faces"
for filename in os.listdir(path):
    img_path = os.path.join(path, filename)
    name = os.path.splitext(filename)[0]
    img = face_recognition.load_image_file(img_path)
    encoding = face_recognition.face_encodings(img)
    if encoding:
        known_face_encodings.append(encoding[0])
        known_face_names.append(name)

Haar Face Detection

In [10]:
def detect_faces(img):
    face_img = img.copy()
    face_rects = face_cascade.detectMultiScale(face_img, scaleFactor=1.2, minNeighbors=5)
    return face_rects

face_recognition Library for Recognizing New Faces by Comparing with Known Faces

In [11]:
def recognize_faces(img, face_rects):
    face_img = img.copy()
    for (x, y, w, h) in face_rects:
        roi = face_img[y:y+h, x:x+w]
        rgb_face = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        encodings = face_recognition.face_encodings(rgb_face)
        name = "Unknown"
        if encodings:
            matches = face_recognition.compare_faces(known_face_encodings, encodings[0])
            face_distances = face_recognition.face_distance(known_face_encodings, encodings[0])

            if True in matches:
                best_match_index = np.argmin(face_distances)
                name = known_face_names[best_match_index]
        
        cv2.rectangle(face_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(face_img, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
    return face_img

Face Recognition on Still Image

In [14]:
query_img = cv2.imread('content/query_elon.jpg')
face_rects = detect_faces(query_img)
if len(face_rects) > 0:
    recognized_img = recognize_faces(query_img, face_rects)
    cv2.imshow('Recognized Faces', recognized_img)
    cv2.waitKey(0)
cv2.destroyAllWindows()

Real-Time Face Recognition

In [ ]:
cam = cv2.VideoCapture(0)

while True:
    ret, frame = cam.read()
    if not ret:
        break
    
    face_rects = detect_faces(frame)
    result = recognize_faces(frame, face_rects)
    cv2.imshow("Face Recognition", result)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cam.release()
cv2.destroyAllWindows()


**Real-Time Landmark and Face Recognition**

In [12]:
cam = cv2.VideoCapture(0)

while True:
    ret, frame = cam.read()
    if not ret:
        break
    face_rects = detect_faces(frame)
    frame = recognize_faces(frame, face_rects)
    result = detect_landmark(frame, match_threshold=10)
    cv2.imshow("Landmark Detection", result)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cam.release()
cv2.destroyAllWindows()

✅ Landmark detected with 12 inlier matches.
✅ Landmark detected with 14 inlier matches.
✅ Landmark detected with 13 inlier matches.
✅ Landmark detected with 10 inlier matches.
✅ Landmark detected with 15 inlier matches.
✅ Landmark detected with 15 inlier matches.
✅ Landmark detected with 12 inlier matches.
✅ Landmark detected with 19 inlier matches.
✅ Landmark detected with 12 inlier matches.
✅ Landmark detected with 11 inlier matches.


# **Time of Day Classification**

In [9]:
time_of_day_path = "Images"

Quantitative Evidence

In [10]:
def compute_brightness_contrast_metrics(img):
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mean_rgb = np.mean(rgb)
    contrast_rgb = np.std(rgb)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mean_hsv_v = np.mean(hsv[:,:,2])
    contrast_hsv_v = np.std(hsv[:,:,2])
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    mean_lab_l = np.mean(lab[:,:,0])
    contrast_lab_l = np.std(lab[:,:,0])
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mean_gray = np.mean(gray)
    contrast_gray = np.std(gray)
    return {
        "mean_rgb": mean_rgb, "contrast_rgb": contrast_rgb,
        "mean_hsv_v": mean_hsv_v, "contrast_hsv_v": contrast_hsv_v,
        "mean_lab_l": mean_lab_l, "contrast_lab_l": contrast_lab_l,
        "mean_gray": mean_gray, "contrast_gray": contrast_gray
    }

# Store results
results = []
for filename in os.listdir(time_of_day_path):
    img_path = os.path.join(time_of_day_path, filename)
    img = cv2.imread(img_path)
    metrics = compute_brightness_contrast_metrics(img)
    metrics["filename"] = filename
    results.append(metrics)

In [11]:
print(f"{'Filename':<20} {'RGB':>8} {'C_RGB':>8} {'HSV_V':>8} {'C_HSV':>8} {'LAB_L':>8} {'C_LAB':>8} {'GRAY':>8} {'C_GRAY':>8}")
for r in results:
    print(f"{r['filename']:<20} {r['mean_rgb']:8.2f} {r['contrast_rgb']:8.2f} {r['mean_hsv_v']:8.2f} {r['contrast_hsv_v']:8.2f} {r['mean_lab_l']:8.2f} {r['contrast_lab_l']:8.2f} {r['mean_gray']:8.2f} {r['contrast_gray']:8.2f}")

Filename                  RGB    C_RGB    HSV_V    C_HSV    LAB_L    C_LAB     GRAY   C_GRAY
Image_1.jpg            139.26    59.88   163.17    62.14   147.26    50.40   138.39    50.16
Image_10.jpg            59.60    47.00    73.82    50.55    61.85    48.18    59.34    44.69
Image_11.jpg            66.06    47.73    86.26    57.39    72.78    50.15    69.00    45.41
Image_12.jpg            50.88    45.92    61.78    52.05    51.51    47.54    50.44    44.03
Image_13.jpg            52.01    51.04    66.74    59.61    56.07    55.33    54.06    50.72
Image_14.jpg            49.16    37.21    64.56    42.44    49.37    37.87    47.99    34.28
Image_15.jpg            45.30    45.51    69.22    51.08    46.86    44.42    44.06    40.04
Image_16.jpg            31.28    30.99    34.21    32.77    29.96    33.41    31.44    30.95
Image_2.jpg            157.40    70.69   171.66    70.19   160.98    69.32   155.42    68.53
Image_3.jpeg           181.98    58.56   197.74    51.69   185.65    5

Classification thresholds

In [12]:
thresholds = {
    "mean_hsv_v": 100,
    "mean_lab_l": 110,
    "mean_rgb": 120,
    "mean_gray": 100
}

In [16]:
def classify_and_display(img, filename, metric="mean_hsv_v"):
    metrics = compute_brightness_contrast_metrics(img)
    value = metrics[metric]
    threshold = thresholds[metric]
    label = "Day" if value > threshold else "Night"
    print(f"\n{filename} - {metric}: {value:.2f} - Prediction: {label}")
    display_img = img.copy()
    cv2.putText(display_img, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)
    cv2.imshow(f"{filename} - {label}", display_img)
    print("Press any key in the image window to close it.")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [17]:
print("\nAvailable metrics for classification:")
print("1. mean_hsv_v (recommended)")
print("2. mean_lab_l")
print("3. mean_rgb")
print("4. mean_gray")
metric_map = {
    "1": "mean_hsv_v",
    "2": "mean_lab_l",
    "3": "mean_rgb",
    "4": "mean_gray"
}
user_metric = input("Choose metric (1-4): ").strip()


Available metrics for classification:
1. mean_hsv_v (recommended)
2. mean_lab_l
3. mean_rgb
4. mean_gray


In [26]:
metric = metric_map.get(user_metric, "mean_hsv_v")

for filename in os.listdir(time_of_day_path):
    img_path = os.path.join(time_of_day_path, filename)
    img = cv2.imread(img_path)
    classify_and_display(img, filename, metric)
    


Image_1.jpg - mean_hsv_v: 163.17 - Prediction: Day
Press any key in the image window to close it.

Image_10.jpg - mean_hsv_v: 73.82 - Prediction: Night
Press any key in the image window to close it.

Image_11.jpg - mean_hsv_v: 86.26 - Prediction: Night
Press any key in the image window to close it.

Image_12.jpg - mean_hsv_v: 61.78 - Prediction: Night
Press any key in the image window to close it.

Image_13.jpg - mean_hsv_v: 66.74 - Prediction: Night
Press any key in the image window to close it.

Image_14.jpg - mean_hsv_v: 64.56 - Prediction: Night
Press any key in the image window to close it.

Image_15.jpg - mean_hsv_v: 69.22 - Prediction: Night
Press any key in the image window to close it.

Image_16.jpg - mean_hsv_v: 34.21 - Prediction: Night
Press any key in the image window to close it.

Image_2.jpg - mean_hsv_v: 171.66 - Prediction: Day
Press any key in the image window to close it.

Image_3.jpeg - mean_hsv_v: 197.74 - Prediction: Day
Press any key in the image window to close

# **Image Quality Assessment**

In [3]:
path = "IQA"

In [4]:
def wrap_text(text, font, font_scale, max_width):
    words = text.split(' ')
    lines = []
    current_line = ""
    
    for word in words:
        test_line = current_line + (" " if current_line else "") + word
        text_size = cv2.getTextSize(test_line, font, font_scale, 1)[0]
        
        if text_size[0] <= max_width:
            current_line = test_line
        else:
            if current_line:
                lines.append(current_line)
                current_line = word
            else:
                lines.append(word)
    
    if current_line:
        lines.append(current_line)
    
    return lines

def draw_wrapped_text(img, text, start_pos, font, font_scale, color, thickness, max_width):
    lines = wrap_text(text, font, font_scale, max_width)
    x, y = start_pos
    line_height = cv2.getTextSize("Ay", font, font_scale, thickness)[0][1] + 10
    
    for i, line in enumerate(lines):
        cv2.putText(img, line, (x, y + i * line_height), font, font_scale, color, thickness)
    
    return y + len(lines) * line_height

Detect Blur

In [5]:
def is_blur(img):
    threshold=100
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    variance = cv2.Laplacian(gray, cv2.CV_64F).var()
    return variance < threshold, variance

Highlight Blur

In [6]:
def highlight_blur_regions(img, gray, threshold=100):
    height, width = gray.shape
    window_size = min(height, width) // 8
    if window_size < 50:
        window_size = 50
    
    overlay = img.copy()
    
    for y in range(0, height - window_size, window_size // 2):
        for x in range(0, width - window_size, window_size // 2):
            window = gray[y:y+window_size, x:x+window_size]
            laplacian_var = cv2.Laplacian(window, cv2.CV_64F).var()
            
            if laplacian_var < threshold:
                cv2.rectangle(overlay, (x, y), (x+window_size, y+window_size), (0, 0, 255), -1)
    return cv2.addWeighted(img, 0.8, overlay, 0.2, 0)

In [7]:
detect_blur_path = "IQA/detect_blur"
for filename in os.listdir(detect_blur_path):
    img_path = os.path.join(detect_blur_path, filename)
    img = cv2.imread(img_path)

    height, width = img.shape[:2]
    max_text_width = width - 20
    
    text = "Not Blurry"
    blurred, variance = is_blur(img)
    
    if blurred:
        text = "Blurry"
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = highlight_blur_regions(img, gray)

    cv2.putText(img, f"{text}: {variance:.2f}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    
    if blurred:
        enhancement_text = "Enhancement: Apply sharpening filter (Laplacian) or recapture with stable camera."
        draw_wrapped_text(img, enhancement_text, (10, 70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, max_text_width)
    
    cv2.imshow("Image Quality Assessment - Blur", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Detect Exposure

In [27]:
def classify_exposure(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
    total_pixels = gray.size
    dark_pixels = np.sum(hist[:50])
    bright_pixels = np.sum(hist[205:])

    dark_ratio = dark_pixels / total_pixels
    bright_ratio = bright_pixels / total_pixels

    if dark_ratio > 0.3:
        return "Underexposed"
    elif bright_ratio > 0.3:
        return "Overexposed"
    else:
        return "Well-exposed"

Highlight Exposure

In [26]:
def highlight_exposure_regions(img, exposure_type):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    overlay = img.copy()
    
    if exposure_type == "Underexposed":
        mask = gray < 50
        overlay[mask] = [0, 255, 255]
    elif exposure_type == "Overexposed":
        mask = gray > 205
        overlay[mask] = [0, 165, 255]
    
    return cv2.addWeighted(img, 0.8, overlay, 0.2, 0)

In [39]:
detect_exposure_path = "IQA/detect_exposure"
for filename in os.listdir(detect_exposure_path):
    img_path = os.path.join(detect_exposure_path, filename)
    img = cv2.imread(img_path)
    height, width = img.shape[:2]
    max_text_width = width - 20

    exposure = classify_exposure(img)

    cv2.putText(img, f"{exposure}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    
    if exposure in ["Underexposed", "Overexposed"]:
        img = highlight_exposure_regions(img, exposure)
        
        if exposure == "Underexposed":
            enhancement_text = "Enhancement: Increase brightness and adjust levels/curves to recover shadow detail."
        else:  
            enhancement_text = "Enhancement: Decrease brightness and adjust levels/curves to recover highlight detail."
        
        draw_wrapped_text(img, enhancement_text, (10, 70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, max_text_width)   
    cv2.imshow("Image Quality Assessment - Exposure", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Detect Contrast

In [ ]:
%pip install scikit-image

In [29]:
from skimage.exposure import is_low_contrast

Highlight Contrast

In [34]:
def highlight_low_contrast_regions(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    height, width = gray.shape
    window_size = min(height, width) // 6
    if window_size < 50:
        window_size = 50
    
    overlay = img.copy()
    
    for y in range(0, height - window_size, window_size // 2):
        for x in range(0, width - window_size, window_size // 2):
            window = gray[y:y+window_size, x:x+window_size]
            if is_low_contrast(window, 0.3):
                cv2.rectangle(overlay, (x, y), (x+window_size, y+window_size), (255, 0, 0), -1)
    
    return cv2.addWeighted(img, 0.8, overlay, 0.2, 0)

In [ ]:
detect_contrast_path = "IQA/detect_contrast"
for filename in os.listdir(detect_contrast_path):
    img_path = os.path.join(detect_contrast_path, filename)
    img = cv2.imread(img_path)
    height, width = img.shape[:2]
    max_text_width = width - 20
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    text_contrast = "Normal Contrast"
    enhancement_text_contrast = None

    if is_low_contrast(gray, 0.35):
        text_contrast = "Low Contrast"
        enhancement_text_contrast = "Enhancement: Increase contrast using curves, levels, or histogram equalization."
        img = highlight_low_contrast_regions(img)
    elif not is_low_contrast(gray, 0.85):
        text_contrast = "High Contrast"
        enhancement_text_contrast = "Enhancement: Decrease contrast using curves or reducing highlights/shadows."

    cv2.putText(img, f"Contrast: {text_contrast}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    if enhancement_text_contrast:
        draw_wrapped_text(img, enhancement_text_contrast, (10, 70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, max_text_width)

    cv2.imshow("Image Quality Assessment - Contrast", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Detect Noise

In [13]:
def detect_noise(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5,5), 0)
    diff = cv2.absdiff(gray, blurred)

    mean, std = np.mean(diff), np.std(diff)

    if mean > 10 or std > 20:
        return True
    return False

Highlight Noise

In [35]:
def highlight_noisy_regions(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    height, width = gray.shape
    window_size = min(height, width) // 8
    if window_size < 50:
        window_size = 50
    
    overlay = img.copy()
    
    for y in range(0, height - window_size, window_size // 2):
        for x in range(0, width - window_size, window_size // 2):
            window = gray[y:y+window_size, x:x+window_size]
            blurred_window = cv2.GaussianBlur(window, (5, 5), 0)
            diff = cv2.absdiff(window, blurred_window)
            
            mean_diff = np.mean(diff)
            std_diff = np.std(diff)
            
            if mean_diff > 8 or std_diff > 15:
                cv2.rectangle(overlay, (x, y), (x+window_size, y+window_size), (128, 0, 128), -1)
    
    return cv2.addWeighted(img, 0.8, overlay, 0.2, 0)

In [ ]:
detect_noise_path = "IQA/detect_noise"
for filename in os.listdir(detect_noise_path):
    img_path = os.path.join(detect_noise_path, filename)
    img = cv2.imread(img_path)
    height, width = img.shape[:2]
    max_text_width = width - 20
    
    noise_detected = detect_noise(img)
    text_noise = "Not Noisy"
    enhancement_text_noise = None

    if noise_detected:
        text_noise = "Noisy"
        enhancement_text_noise = "Enhancement: Apply noise reduction filter (Gaussian, median, or bilateral filter)."
        img = highlight_noisy_regions(img)

    cv2.putText(img, f"Noise: {text_noise}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    if enhancement_text_noise:
        draw_wrapped_text(img, enhancement_text_noise, (10, 70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, max_text_width)
    
    cv2.imshow("Image Quality Assessment - Noise", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# **Similarity Retrieval**

In [24]:
query_path = "content/query.jpg"
landmark_dir = "content/museum_images"

sift = cv2.SIFT_create()
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

landmark_descriptors = []
landmark_keypoints = []
landmark_images = []
landmark_names = []

for img_name in os.listdir(landmark_dir):
    img_path = os.path.join(landmark_dir, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue
    img = cv2.resize(img, (600, 400))
    kp, des = sift.detectAndCompute(img, None)
    if des is not None:
        landmark_keypoints.append(kp)
        landmark_descriptors.append(des)
        landmark_images.append(img)
        landmark_names.append(img_name)

def retrieve_top_similar_images(query_img, top_k=3, match_threshold=10):
    query_img = cv2.resize(query_img, (600, 400))
    query_kp, query_des = sift.detectAndCompute(query_img, None)
    results = []

    for i, des in enumerate(landmark_descriptors):
        matches = bf.match(des, query_des)
        matches = sorted(matches, key=lambda x: x.distance)
        good_matches = matches[:50]

        if len(good_matches) >= match_threshold:
            src_pts = np.float32([landmark_keypoints[i][m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([query_kp[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
            if mask is not None:
                inliers = int(np.sum(mask))
                results.append((inliers, i, good_matches))

    results.sort(reverse=True, key=lambda x: x[0])
    top_images = []
    for inliers, idx, good_matches in results[:top_k]:
        matched_img = cv2.drawMatches(
            landmark_images[idx], landmark_keypoints[idx],
            query_img, query_kp, good_matches, None,
            matchColor=(0, 255, 0), flags=2
        )
        top_images.append((landmark_names[idx], matched_img, inliers))
    return top_images

In [27]:
query_image = cv2.imread(query_path, cv2.IMREAD_GRAYSCALE)
top_matches = retrieve_top_similar_images(query_image)

for i, (name, img, inliers) in enumerate(top_matches):
    print(f"Match {i+1}: {name} with {inliers} inliers")
    cv2.imshow(f"Top Match {i+1}", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Match 1: Museum of Future 6.jpg with 32 inliers
Match 2: Museum of Future 5.jpg with 8 inliers
Match 3: Museum of Future 1.jpeg with 7 inliers


# **Annotation Capability**

Enhanced as per Ms Asma's suggestions during the presentation

Drawing Rectangle

In [2]:
def draw_rectangle(event, x, y, flags, param):
    global drawing, start_x, start_y

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        start_x, start_y = x, y
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()
            cv2.rectangle(img_copy, (start_x, start_y), (x, y), (0, 255, 0), 2)
            cv2.imshow("image", img_copy)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.rectangle(img, (start_x, start_y), (x, y), (0, 255, 0), 2)
        cv2.imshow("image", img)

Drawing Circle

In [3]:
def draw_circle(event, x, y, flags, param):
    global drawing, start_x, start_y

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        start_x, start_y = x, y
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()
            center_x = (start_x + x) // 2
            center_y = (start_y + y) // 2
            radius = int(0.5 * np.sqrt((x - start_x)**2 + (y - start_y)**2))
            cv2.circle(img_copy, (center_x, center_y), radius, (0, 0, 255), 2)
            cv2.imshow("image", img_copy)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        center_x = (start_x + x) // 2
        center_y = (start_y + y) // 2
        radius = int(0.5 * np.sqrt((x - start_x)**2 + (y - start_y)**2))
        cv2.circle(img, (center_x, center_y), radius, (0, 0, 255), 2)
        cv2.imshow("image", img)

Drawing Line

In [4]:
def draw_line(event, x, y, flags, parmas):
    global drawing, start_x, start_y

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        start_x, start_y = x, y
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()
            cv2.line(img_copy, (start_x, start_y), (x, y), (255, 0, 0), 2)
            cv2.imshow("image", img_copy)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.line(img, (start_x, start_y), (x, y), (255, 0, 0), 2)
        cv2.imshow("image", img)

Writing Text

In [5]:
def write_text(event, x, y, flags, param):
    global img
    if event == cv2.EVENT_LBUTTONDOWN:
        text = input("Enter text to write: ")
        cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("image", img)

Main Annotation Logic

In [76]:
# Global variables
drawing = False
start_x, start_y = -1, -1
current_mode = 1  # 1: Circle, 2: Rectangle, 3: Line, 4: Text


def mouse_callback(event, x, y, flags, param):
    global drawing, start_x, start_y, current_mode, img
    
    if current_mode == 1:  # Circle
        draw_circle(event, x, y, flags, param)
    elif current_mode == 2:  # Rectangle
        draw_rectangle(event, x, y, flags, param)
    elif current_mode == 3:  # Line
        draw_line(event, x, y, flags, param)
    elif current_mode == 4:  # Text
        write_text(event, x, y, flags, param)


img = cv2.imread("content/starry_night.jpg")

original_img = img.copy()

cv2.namedWindow("image")
cv2.setMouseCallback("image", mouse_callback)

print("Press keys to switch modes:")
print("1 - Circle mode (Red)")
print("2 - Rectangle mode (Green)")  
print("3 - Line mode (Blue)")
print("4 - Text mode (Green) - Check console for input")
print("ESC - Exit")
print("s - Save image")
print("r - Reset image")
print('\n')

while True:
    cv2.imshow("image", img)
    key = cv2.waitKey(1) & 0xFF
    
    if key == 27:
        break
    elif key == ord('1'):
        current_mode = 1
        print("Switched to Circle mode (Red)")
    elif key == ord('2'):
        current_mode = 2
        print("Switched to Rectangle mode (Green)")
    elif key == ord('3'):
        current_mode = 3
        print("Switched to Line mode (Blue)")
    elif key == ord('4'):
        current_mode = 4
        print("Switched to Text mode (Green)")
    elif key == ord('s'):
        cv2.imwrite("drawn_image.jpg", img)
        print("Image saved as 'drawn_image.jpg'")
    elif key == ord('r'):
        img = original_img.copy()
        cv2.imshow("image", img)
        print("Image reset to original")

cv2.destroyAllWindows()



Press keys to switch modes:
1 - Circle mode (Red)
2 - Rectangle mode (Green)
3 - Line mode (Blue)
4 - Text mode (Green) - Check console for input
ESC - Exit
s - Save image
r - Reset image


Switched to Rectangle mode (Green)
Switched to Line mode (Blue)
Switched to Text mode (Green)
